In [ ]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
import requests
from MCForecastTools import MCSimulation
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [ ]:
# Load .env enviroment variables
load_dotenv("ProjectAPI.env")

In [ ]:
# Set the tickers
tickers = ["SPY", "IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]

# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("Alpaca_API_Key")
alpaca_secret_key = os.getenv("Alpaca_Secret_Key")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

In [ ]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp("2022-04-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-10-01", tz="America/New_York").isoformat()

In [ ]:
# Set timeframe to 18 months for the Alpaca API
timeframe = "1Day"

# Get 12 months of price data for etf portfolio
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio

In [ ]:
#Drop unnecessary columns for our portfolio dataframe
df_portfolio.drop(columns=['high','low','trade_count','vwap'], axis=1, inplace=True)

#Rearrange columns
df_portfolio = df_portfolio[['symbol', 'close', 'open', 'volume']]

df_portfolio

In [ ]:
# Reorganize the DataFrame and separate ticker data
# Order:  SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI

# Separate ticker data
SPY = df_portfolio[df_portfolio["symbol"]=="SPY"].drop("symbol", axis=1)
EWA = df_portfolio[df_portfolio["symbol"]=="EWA"].drop("symbol", axis=1)
EWC = df_portfolio[df_portfolio["symbol"]=="EWC"].drop("symbol", axis=1)
EWJ = df_portfolio[df_portfolio["symbol"]=="EWJ"].drop("symbol", axis=1)
EWU = df_portfolio[df_portfolio["symbol"]=="EWU"].drop("symbol", axis=1)
EWZ = df_portfolio[df_portfolio["symbol"]=="EWZ"].drop("symbol", axis=1)
EWW = df_portfolio[df_portfolio["symbol"]=="EWW"].drop("symbol", axis=1)
EZU = df_portfolio[df_portfolio["symbol"]=="EZU"].drop("symbol", axis=1)
IWM = df_portfolio[df_portfolio["symbol"]=="IWM"].drop("symbol", axis=1)
MCHI = df_portfolio[df_portfolio["symbol"]=="MCHI"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
df_portreo = pd.concat([SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI], axis=1, keys=['SPY', 'EWA', 'EWC', 'EWJ', 'EWU', 'EWZ', 'EWW', 'EZU', 'IWM', 'MCHI'])

# Display sample data
df_portreo

In [ ]:
#Export clean dataframe to .csv file for backup
#df_portfolio.to_csv('UpDownFlat.csv')

df_portreo.to_csv(r'Desktop\UpDownFlatData.csv')

In [ ]:
#Create the Monte Carlo Simulation for the next day
MC_df = df_portreo["SPY", "IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]

monte_predict = MCSimulation(
    portfolio_data = MC_df,
    weights = [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
    num_simulation = 1000,
    num_trading_days = 1
)

#Check it worked
MC_df.portfolio_data.head()

In [ ]:
# Run a Monte Carlo simulation to forecast tomorrow's movement
MC_df.calc_cumulative_return()

In [ ]:
#Show what the prediction average and std are to give an idea of tomorrow's likelyhood of up/down
MC_tbl = MC_df.summarize_cumulative_return()
MC_tbl

In [ ]:
#based on the table output, display the mean and 1.5 std up and down to show the target outcomes.


In [ ]:
#define variables for regression

X=df_portfolio[["IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]]

y=df_portfolio['SPY']

In [ ]:
#set up test and train section of the df
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [ ]:
#run regression and predict
reg = LinearRegression()
 
# Fit the model to the data
reg_model = LinearRegression().fit(X_train, y_train)

# Print the coefficients of the model to determine influences on X
coeffs = reg_model.coef_
ticks = ["IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]
count = 0
for t in coeffs:
    print(f"{ticks[count]} beta is {round(coeffs[count],4)}")
    count +=1

In [ ]:
#Predicting the Test and Train set result 

y_pred= reg_model.predict(X_test)  
x_pred= reg_model.predict(X_train)

#finding the difference between predicted and actual values- for reference of predictive accuracy of model
reg_model_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred})
reg_model_diff.plot(figsize=(20,10))

In [ ]:
#display the regression output into the GUI for analysis
